# Czym jest owlready2

Pakiet Pythona pomocny w pracy z ontologiami reprezentowanymi w języku OWL. Umożliwia:

- programowanie obiektowe ukierunkowane na ontologie 
- wykorzystanie ekspresyjności ontologii w zakresie języka OWL
- szybkie ładowanie bardzo dużych ontologii (dziesiątki lub setki gigabajtów)
 

Owlready zawiera wewnętrzną grafową bazę (*quadstore*), która uwzględnia semantykę OWL. Baza zawiera czwórki w formacie RDF, gdzie do RDFowej trójki dodawany jest identyfikator ontologii.
Ta baza czwórek może być umieszczona w pamięci RAM, na dysku lub w formie pliku bazy danych SQLite3.
OWlready ładuje encje ontologii do Pyhtona na żądanie, wtedy kiedy są używane.
Jeśli encje zostaną zmodyfikowane w Pythonie, owlready automatycznie aktualizuje bazę czwórek.

# Przykład - ontologia zwierząt domowych

Żeby zilustrować możliwości Owlready2 zajmiemy się prostym przykładem  ontologii o zwierzętach domowych.
Dla przykładu, uwzględnimy charakterystyki takie jak:
1.   Pochodzenie: z hodowli, ze schroniska.
2.   Spożywany pokarm: roślinny, zwierzęcy.
3.   Rozmiar, waga, itp.

# Dostęp do ontologii w Pythonie

## Import owlready
Zaimportujmy owlready (wersja 2) do Pythona:










In [1]:
!pip install owlready2
from owlready2 import *

## Ładowanie ontologii
Owlready pozwala załadować ontologię OWL do Pythona i mieć dostęp do encji ontologii tak jak do obiektów w module Pythona. 
Ontologię można załadować na trzy różne sposoby:

1. Poprzez IRI (adres w Internecie).
2. Z lokalnego pliku.
3. Z obiektu pliku poprzez `open()`, `urlopen()` i inne funkcje.

Obsługiwane formaty plików to: RDF/MXL, OWL/XML i N-Triples.

Załadujmy ontologię z lokalnego pliku:






In [2]:
onto = get_ontology("zwierzeta.owl").load()

Atrybut `base_iri` ontologii pozwala pozyskać jej IRI:

In [3]:
print(onto.base_iri)

http://www.semanticweb.org/ontologies/zwierzeta#


## Listowanie zawartości ontologii

Obiekt ontologii zawiera wiele metod do poruszania się po encjach zawartych w ontologii, w zależności od ich typu.
Te metody zwracają generatory. Żeby wyświetlić zawartość możemy użyć funkcji Pythona `list()`, która konwertuje generator na listę.
Przykładowo, wyświetlmy klasy ontologii:


In [4]:
print(onto.classes())
list(onto.classes())

<generator object _GraphManager.classes at 0x7f0e5e3adeb0>


[zwierzeta.Charakterystyka,
 zwierzeta.Chomik,
 zwierzeta.Hodowla,
 zwierzeta.Kot,
 zwierzeta.Królik,
 zwierzeta.MałeZwierzę,
 zwierzeta.Owłosienie,
 zwierzeta.Pies,
 zwierzeta.Pochodzenie,
 zwierzeta.Pokarm,
 zwierzeta.PokarmRoślinny,
 zwierzeta.PokarmZwierzęcy,
 zwierzeta.Ptak,
 zwierzeta.Rybka,
 zwierzeta.Schronisko,
 zwierzeta.Waga,
 zwierzeta.Wszystkożerca,
 zwierzeta.ZwierzęDomowe,
 zwierzeta.ŚwinkaMorska]

Jednak lepiej nie używać `list()` gdy generator jest obecny w pętli, w celu polepszenia wydajności:


In [5]:
for c in onto.classes(): print (c.name)

Charakterystyka
Chomik
Hodowla
Kot
Królik
MałeZwierzę
Owłosienie
Pies
Pochodzenie
Pokarm
PokarmRoślinny
PokarmZwierzęcy
Ptak
Rybka
Schronisko
Waga
Wszystkożerca
ZwierzęDomowe
ŚwinkaMorska


<span style="color:red"> __Zadanie 1: Wylistuj własności w ontologii, używając do tego celu metody `properties()` __ </span>

In [6]:
for x in onto.properties():
    print(x)

zwierzeta.ma_wartość
zwierzeta.je
zwierzeta.ma_charakterystykę
zwierzeta.ma_część
zwierzeta.ma_funkcję
zwierzeta.pochodzi_z


## Dostęp do encji

Obiekty Pythona służące do dostępu do encji są tworzone dynamicznie, na żądanie. 
Pseudo-słownik IRIS pozwala na dostęp do każdej encji poprzez jej IRI. Np. żeby aby pozyskać dostęp do encji "kot" możemy użyć jej całego IRI albo skróconej wersji notacji:


In [7]:
#dostęp do encji poprzez IRIS
print(IRIS["http://www.semanticweb.org/ontologies/zwierzeta#Kot"])

#dostęp do encji poprzez skrócony zapis
print(onto.Kot)

zwierzeta.Kot
zwierzeta.Kot


Jeśli nazwa (etykieta) encji zawiera znaki nie wspierane przez Pythona, można użyć też składni:

In [8]:
onto["Kot"]

zwierzeta.Kot

__Zadanie 2: Wyświetl serializację obiektu o etykiecie 'Chomik' __

In [9]:
onto.Chomik

zwierzeta.Chomik

### Indywidua

Indywiduami można manipulować tak jakby były zwykłymi obiektami Pythona. Można testować ich przynależność do klasy za pomocą funkcji `isinstance()`, tak jak w przypadku dowolnych obiektów Pythona.

In [10]:
isinstance(onto.Kajtek, onto.Kot)

True

### Klasy
Dostęp do klas powiązanych z daną klasą możemy uzyskać poprzez notację z '.'. Klasy ontologii są prawdziwymi klasami w Pythonie i mogą być tak używane. 
Np. funkcja `issubclass()` testuje czy klasa jest potomkiem (podklasą, podklasą podklasy itp.) innej klasy:

In [11]:
issubclass(onto.Kot, onto.ZwierzęDomowe)

True

Najlepiej jest jednak użyć atrybutu `is_a`, który także zawiera ograniczenia i konstruktory logiczne.

In [12]:
onto.Kot.is_a

[zwierzeta.ZwierzęDomowe]

Możemy także użyć `is_a` aby odpytać o klasy danego indywiduum.

__Zadanie 3: Użyj `is_a` aby wyświetlić listę klas, do których przynależy indywiduum `Kajtek` __



In [13]:
onto.Kajtek.is_a

[zwierzeta.Kot]

Metoda `subclasses()` zwraca listę klasy dzieci (pamiętajmy, że jest to w formie generatora)

In [14]:
list(onto.ZwierzęDomowe.subclasses())

[zwierzeta.Kot,
 zwierzeta.MałeZwierzę,
 zwierzeta.Pies,
 zwierzeta.Ptak,
 zwierzeta.Rybka]

Metody `ancestors()` i `descendants()` są używane do pozyskania zbioru przodków i potomków.

__Zadanie 4: Wyświetl przodków oraz potomków obiektu o etykiecie 'MałeZwierzę' __

In [15]:
print(onto.MałeZwierzę.ancestors())
print(onto.MałeZwierzę.descendants())

{zwierzeta.ZwierzęDomowe, zwierzeta.MałeZwierzę, owl.Thing}
{zwierzeta.MałeZwierzę, zwierzeta.Królik, zwierzeta.ŚwinkaMorska, zwierzeta.Chomik}


Możemy także chcieć pozyskać dostęp do klas, które są semantycznie równoważne:


In [16]:
onto.Wszystkożerca.equivalent_to

[zwierzeta.je.some(zwierzeta.PokarmRoślinny) & zwierzeta.je.some(zwierzeta.PokarmZwierzęcy)]

### Relacje
Relacje (właściwości) poprzez jakie jest powiązane z innymi dane indywiduum można pozyskać używając notacji "individual.attribute", np.:

In [17]:
onto.Figaro.ma_charakterystykę

[zwierzeta.wagaFigaro]

__Zadanie 5: Wyświetl wartość relacji (właściwości) ma_wartość z jaką jest powiązana instancja 'wagaFigaro' __

In [18]:
onto.wagaFigaro.ma_wartość

[5]

# Tworzenie i modyfikowanie ontologii
## Tworzenie pustej ontologii
Funkcja `get_ontology()` pozwala na utworzenie pustej ontologii na bazie IRI. Preferowany jest separator "#" lub "/" na końcu IRI, ponieważ owlready nie może się ich "domyślić" z uwagi na to, że ontologia jest pusta.

In [19]:
onto_nowa = get_ontology("http://przyklad.org/onto_nowa.owl#")

##Tworzenie klas
Aby utworzyć klasę, po prostu utwórz klasę Pythona, która dziedziczy z `Thing`.
Przykładowo, możemy utworzyć klasę KotBrytyjski w następujący sposób:

In [20]:
with onto:
  class KotBrytyjski(Thing): pass

Żeby obserwować co dzieje się w repozytorium "czwórek" owlready, możemy użyć funkcji `set_log_level()`. Poniżej znajduje się przykład ustwaienia poziomo logowania na 9 (najwyższy poziom). Możemy teraz obserwować jakie trójki RDF są dodawane, usuwane, modyfikowane.

In [21]:
set_log_level(9)
with onto:
  class KotSyberyjski(Thing): pass

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#KotSyberyjski http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#KotSyberyjski http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.w3.org/2002/07/owl#Thing


__Zadanie 6: Utwórz nową klasę 'KotPerski' jako podklasę klasy 'Kot' __


In [22]:
with onto:
    class KotPerski(onto.Kot): pass

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#KotPerski http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#KotPerski http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/ontologies/zwierzeta#Kot


Klasy możemy tworzyć w sposób dynamiczny, wtedy kiedy nie znamy nazwy klasy w trakcie tworzenia programu, ale jest dostępna w trakcie jego działania.

In [23]:
import types
nazwa_klasy = "NowaKlasa"
Nadklasy = [Thing, onto.Kot]
with onto:
  NowaKlasa = types.new_class(nazwa_klasy, tuple (Nadklasy))

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#NowaKlasa http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#NowaKlasa http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/ontologies/zwierzeta#Kot
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#NowaKlasa http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.w3.org/2002/07/owl#Thing


Zwróć uwagę, że w powyższym przykładzie zostało także wykorzystane wielokrotne dziedziczenie.

## Tworzenie właściwości
W owlreaady właściwości są traktowane podobnie do klas (szczególnie jeśli chodzi o wsparcie dziedziczenia).
Właściwości są pewnego rodzaju "klasami relacji". 
Właściwości są tworzone poprzez definiowanie klasy, która dziedziczy z `DataProperty`, `ObjectProperty` lub `AnnotationProperty`. 
Dodatkowo klasy oznaczające charakterystyki właściwości takiej jak `FunctionalProperty`, `InverseFunctionalProperty`, `TransitiveProperty`, `SymmetricProperty` itd. mogą być użyte jako dodatkowe nadklasy.
Atrybuty klasy `domain` i `range` są wykorzystywane aby odpytywać lub definiować dziedzinę i przeciwdziedzinę właściwości, w formie listy.
Przykładowo, możemy chcieć utworzyć właściwość funkcyjną `typ_owłosienia`:

In [24]:
with onto:
   class typ_owlosienia(ObjectProperty, FunctionalProperty):
      domain = [onto['ZwierzęDomowe']]
      range = [onto['Owłosienie']]

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#typ_owlosienia http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#typ_owlosienia http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#FunctionalProperty
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#typ_owlosienia http://www.w3.org/2000/01/rdf-schema#domain http://www.semanticweb.org/ontologies/zwierzeta#ZwierzęDomowe
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#typ_owlosienia http://www.w3.org/2000/01/rdf-schema#range http://www.semanticweb.org/ontologies/zwierzeta#Owłosienie


__Zadanie 7: Utwórz właściwość (`DataProperty`) o nazwie `przyjazne_dla_dzieci`, która jako dziedzinę ma 'ZwierzęDomowe' a jako wartość odpowiedni typ z xml schema reprezentujący wartości boolowskie (**True**, **False**) _

In [25]:
with onto:
    class przyjazne_dla_dzieci(DataProperty):
        domain = [onto.ZwierzęDomowe]
        range = [bool]

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#przyjazne_dla_dzieci http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#DatatypeProperty
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#przyjazne_dla_dzieci http://www.w3.org/2000/01/rdf-schema#domain http://www.semanticweb.org/ontologies/zwierzeta#ZwierzęDomowe
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#przyjazne_dla_dzieci http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#boolean


## Modyfikowanie encji
Relacje pomiędzy indywiduami i ograniczenia egzystencjalne mogą być modyfikowane jak każdy inny atrybut w Pythonie. Przykładowo:
 

In [26]:
onto.wagaFigaro = 4

## Tworzenie encji w ramach przestrzeni nazw
Owlready domyślnie tworzy encje w ramach przestrzeni nazw, tj. IRI encji zaczyna się od IRI ontologii.
Możemy także utworzyć encję z innym IRI, przykładowo:



In [27]:
ro = onto.get_namespace("http://purl.obolibrary.org/obo/")
with ro:
  class ma_rolę(ObjectProperty): pass
ma_rolę.iri


* Owlready2 * ADD TRIPLE http://purl.obolibrary.org/obo/ma_rolę http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty


'http://purl.obolibrary.org/obo/ma_rolę'

## Zmiana nazwy encji
Atrybuty `name` i `iri` danej encji mogą zostać zmodyfikowane (refaktoring):


In [28]:
ma_rolę.iri = "http://www.semanticweb.org/ontologies/zwierzeta#ma_rolę"

## Usuwanie encji
Funkcja globalna destroy_entity pozwala usuwać dowolne encje, np.:




In [29]:
tymczasowe_zwierzę = onto.ZwierzęDomowe()
destroy_entity(tymczasowe_zwierzę)

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#zwierzędomowe1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#zwierzędomowe1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/ontologies/zwierzeta#ZwierzęDomowe


## Usuwanie ontologii
Za pomocą metody `destroy()` można permamentnie usunąć ontologię z repozytorium "czwórek".

## Zapisywanie ontologii
Za pomocą metody `save()` można zapisać ontologię na dysku, np. `onto.save(file)`, gdzie `file` może być nazwą pliku albo obiektem Pythona.

# Konstruktory i ograniczenia 
## Tworzenie konstruktorów
W owlready, ograniczenia tworzone są za pomocą składni "property.restriction_type(value)", np.:


*  property.some(Klasa) dla ograniczeń egzystencjalnych
*  property.only(Klasa) dla ograniczeń uniwersalnych
*  property.value(indywiduum lub dane) dla ograniczenia wartości 
* itd.

Operatory logiczne (NOT, AND, OR) można uzyskać w następujący sposób:


*   Not(Klasa)
*   And([Klasa1, Klasa2, ...]) lub Klasa1 & Klasa2 &...

Konstruktory mogą być wykorzystywane w wyrażeniach z `is_a` i `equivalent_to`. Przykład:







In [30]:
with onto:
  class KotZeSchroniska(onto.Kot):
    equivalent_to = [ onto.Kot & onto.pochodzi_z.some(onto.Schronisko) ] 

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#KotZeSchroniska http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#KotZeSchroniska http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/ontologies/zwierzeta#Kot
* Owlready2 * ADD TRIPLE -17 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Restriction
* Owlready2 * ADD TRIPLE -17 http://www.w3.org/2002/07/owl#onProperty http://www.semanticweb.org/ontologies/zwierzeta#pochodzi_z
* Owlready2 * ADD TRIPLE -17 http://www.w3.org/2002/07/owl#someValuesFrom http://www.semanticweb.org/ontologies/zwierzeta#Schronisko
* Owlready2 * ADD TRIPLE -18 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE -18 http://www.w3.org/2002/07/owl#intersectionOf -16
* Owlready2 * ADD TRIPLE -16 http://www.w3.org/1999/02/22-rdf-synt

__Zadanie 8: Bazując na wynikach poprzednich zadań, utwórz klasę ZwierzęDomoweKrótkowłose, poprzez dodanie klasy 'Krótkowłose' jako podklasy klasy 'Owłosienie' i odpowiedniego ograniczenia wyrażającego to, że ZwierzęDomoweKrótkowłose ma typ owłosienia 'Krótkowłose' _

In [31]:
with onto:
    class Krótkowłose(onto.Owłosienie):
        pass
    
    class ZwierzęDomoweKrótkowłose(onto.ZwierzęDomowe):
        equivalent_to = [onto.ZwierzęDomowe & onto.typ_owlosienia.some(onto.Krótkowłose)]

* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#Krótkowłose http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#Krótkowłose http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/ontologies/zwierzeta#Owłosienie
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#ZwierzęDomoweKrótkowłose http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE http://www.semanticweb.org/ontologies/zwierzeta#ZwierzęDomoweKrótkowłose http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/ontologies/zwierzeta#ZwierzęDomowe
* Owlready2 * ADD TRIPLE -21 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Restriction
* Owlready2 * ADD TRIPLE -21 http://www.w3.org/2002/07/owl#onProperty http://www.semanticweb.org/ontologies/zwierzeta#typ_owlosien

# Automatyczne wnioskowanie
Funkcja `sync_reasoner()` pozwala na uruchomienie silnika wnioskującego i wydedukowaniu nowych faktów do repozytorium "czwórek".
Domyślnie używany jest silnik HermiT. Możliwe jest także użycie silnika Pellet poprzez wywołanie odpowiedniej funkcji: `sync_reasoner_pellet()` (dla silnika HermiT: `sync_reasoner_hermit()`).

In [32]:
sync_reasoner()

* Owlready2 * Creating new ontology inferrences <http://inferrences/>.
* Owlready2 * ADD TRIPLE http://inferrences http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 * Saving world <owlready2.namespace.World object at 0x7f0e5e3bdd60> to /tmp/tmps65iy3ne...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/max/.local/opt/miniconda3/envs/ml/lib/python3.8/site-packages/owlready2/hermit:/home/max/.local/opt/miniconda3/envs/ml/lib/python3.8/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmps65iy3ne
* Owlready2 * HermiT took 0.4487423896789551 seconds
* Owlready * Reparenting zwierzeta.NowaKlasa: {zwierzeta.Kot, owl.Thing} => {zwierzeta.Kot}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
